# Autodesk Moldflow automation tutorial 

Autodesk Moldflow is a popular injection molding software. The Autodesk Moldflow Synergy GUI is able to record user macros for automation as Visual Basic Scripts (*.vbs). However, we may want to integrate this automation with a Python workflow. We could start subprocesses to call Moldflow with a *.vbs script, but it is a much more seamless experience to control Moldflow directly from Python. Luckily, that is possible and this Notebook illustrates how it is done. 

## 1 - Prerequisites 

First of all, you need a functioning Moldflow Insight installation with an active license purchased from [Autodesk](https://www.autodesk.com/products/moldflow/). Note that Moldflow Synergy is available for Microsoft Windows only.

In addition, you'll need a Python installation and must install the pywin32 package via pip or conda to the Python environment:
```
pip install pywin32
conda install pywin32
```

If you want to follow along with this interactive Notebook, make sure to restart the kernel to ensure that `pywin32` is available.

## 2 - Start Moldflow Synergy GUI from Python

Once the prerequisites are met, you can fire up a Moldflow Synergy GUI instance with two simples lines of code. You may execute them directly with this cell:

In [ ]:
# Import win32com.client module from pywin32 to use COM (Component Object Model)
import win32com.client

# Create Moldflow Synergy instance
Synergy = win32com.client.Dispatch("synergy.Synergy")

# Set the units to metric
Synergy.SetUnits("Metric")

This should open a Moldflow Synergy Window on your machine. Pretty cool, isn´t it? In a next step, we learn how to interact with this Window and automate modifications to the model. 

## 3 - Convert VBS to Python Code 

The easiest way to continue from here is the creation of a regular VBS macro in Moldflow Synergy. This is done via the menu `Tools` and the button `Record Macro` in the `Automation` Ribbon. 

Once activated, you can apply modifications in the GUI. Try opening an existing Moldflow project of yours and make some simple modifications like changing the material. After your are finished with the modifications, the recording is stopped with the button `Stop Recording`. A prompt appears to specify a path where to save the *vbs macro. 

This *.vbs file may look like this: 

```vb
'%RunPerInstance
'@ DESCRIPTION
'@ Macro recorded by Synergy on XX-XXX-XXXX at XX:XX:XX
SetLocale("en-us")
Dim SynergyGetter, Synergy
On Error Resume Next
Set SynergyGetter = GetObject(CreateObject("WScript.Shell").ExpandEnvironmentStrings("%SAInstance%"))
On Error GoTo 0
If (Not IsEmpty(SynergyGetter)) Then
	Set Synergy = SynergyGetter.GetSASynergy
Else
	Set Synergy = CreateObject("synergy.Synergy")
End If
Synergy.SetUnits "Metric"
...
```

The first 14 lines of this script are always identical and create a Moldflow Synergy instance. This is equivalent to what we have done in the Python lines in Section 2. To translate the following lines to Python, we need to follow these simple rules: 
1. Place function arguments in regular brackets. 
2. Remove all `Set` statements and trailing regular brackets after referencing objects.
3. Insert a `=`, where values are assigned to variables.

This is illustrated best with a couple of examples:

### 3.1 Import geometries and meshing 

the recorded VBS macro for importing and meshing an STL file looks like this: 

```vb 
'...
Set ImpOpts = Synergy.ImportOptions()
ImpOpts.MeshType "3D"
ImpOpts.Units "mm"
'...
Set Project = Synergy.Project()
Synergy.ImportFile "examples\stl\disk.stl", ImpOpts, False

```

This can be converted to Python like this: 

In [ ]:
# Rule 2
ImpOpts = Synergy.ImportOptions 
# Rule 3
ImpOpts.MeshType = "3D" 
# Rule 3
ImpOpts.Units = "mm" 
# Rule 1
Synergy.ImportFile("examples\stl\disk.stl", ImpOpts, False) 

### 3.2 Changing the material

The recorded VBS macro for changing the molding material looks like this: 

```vb 
'...
Set MatSel = Synergy.MaterialSelector()
MatSel.Select "Covestro.21000.udb", "System", 14261, 0
```

This can be converted to Python like this:

In [ ]:
MatSel = Synergy.MaterialSelector
MatSel.Select("Covestro.21000.udb", "System", 14261, 0)

### 3.3. Create an injection gate at a specific node

The recorded VBS macro for creating an injection gate at a specific node ID looks like this:

```vb 
'...
Set BoundaryConditions = Synergy.BoundaryConditions()
Set Vector = Synergy.CreateVector()
Set EntList = BoundaryConditions.CreateEntityList()
EntList.SelectFromString "N1276 "
Vector.SetXYZ 0, 0, 1
Set EntList_1 = BoundaryConditions.CreateNDBC(EntList, Vector, 40000, Nothing)
```

This can be converted to Python like this:

In [ ]:
import pythoncom 

BoundaryConditions = Synergy.BoundaryConditions
Vector = Synergy.CreateVector
EntList = BoundaryConditions.CreateEntityList
EntList.SelectFromString("N1276 ")
Vector.SetXYZ(0, 0, 1)
EntList_1 = BoundaryConditions.CreateNDBC(EntList, Vector, 40000, pythoncom.Nothing)

Note that we used pythoncom's Nothing object to pass a uninitialized empty object to the function. 

## 4 Known limitations and workarounds 

One issue in automating Moldflow via Python is that some actions (meshing, executing an analysis) call the Moldflow Insight solver. This creates a GUI prompt, where you have to select wether the run should be performed in the cloud, on a remote machine or your local machine. Unfortunately, this prompt cannot be easily answered via the automation script and may be the reason that your script fails to execute completely. However, there are some workarounds:

### 4.1 Meshing

Meshing of CAD geometries typically leads to a stall of your Python code - probably because the CAD kernel needs Moldflow Insight. However, you can work around this issue by importing a triangulated surface mesh format, such as STL. 

### 4.2 Submitting a job 

Submitting a job triggers the mentioned selection prompt. However, you can work around this by saving the model to a *.sdy file and executing this file as a subprocess:

In [ ]:
import subprocess
import os 

# Adapt this path to your system

MOLDFLOW_PATH = "C:/Program Files/Autodesk/Moldflow Insight 2023/bin"

# Save the *.sdy file
StudyDoc = Synergy.StudyDoc
StudyDoc.Save

# Start the study as subprocess
p = subprocess.Popen(
    [os.path.join(MOLDFLOW_PATH, "runstudy.exe"), "disk_study.sdy"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
)
(output, err) = p.communicate()

# Write the output to a log file
with open("disk_study.log", "w") as file:
    file.write(output.decode("windows-1252").strip())